In [2]:
#!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [3]:
import os
import json
from datetime import datetime, timezone, timedelta
import time
import uuid
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import WMLCredentialsCP4D
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Create Dummy Headless ML Provider in OpenScale

This is used when there is no need for OpenScale to communicate with the deployment (e.g., send scoring request to the endpoint).

In [4]:
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest

In [5]:
authenticator = BearerTokenAuthenticator(
        bearer_token=os.environ['USER_ACCESS_TOKEN']
    )

wos_client = APIClient(authenticator=authenticator, 
                       service_url=os.environ['RUNTIME_ENV_APSX_URL'])
wos_client.version

'3.0.14'

In [6]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2022-01-13 06:19:21.327000+00:00,00000000-0000-0000-0000-000000000000


In [7]:
data_marts = wos_client.data_marts.list().result.data_marts
DATA_MART_ID = data_marts[0].metadata.id
DATA_MART_ID 
print("Data Mart ID: {}".format(DATA_MART_ID))

Data Mart ID: 00000000-0000-0000-0000-000000000000


## Create headless service provider

In [ ]:
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"
SERVICE_PROVIDER_DESCRIPTION = ""

In [ ]:
existing_providers_dict = wos_client.service_providers.list().get_result().to_dict()['service_providers']
existing_providers = [sp['entity']['name'] for sp in existing_providers_dict]

if not SERVICE_PROVIDER_NAME in existing_providers:
    added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        operational_space_id = "production",
        credentials=WMLCredentialsCP4D(),
        background_mode=False
     ).result
    
    SERVICE_PROVIDER_ID = added_service_provider_result.metadata.id
    print(f"Service provider ID: {SERVICE_PROVIDER_ID}")
else:
    SERVICE_PROVIDER_ID = next((sp['metadata']['id'] for sp in existing_providers_dict \
                                if sp['entity']['name'] == SERVICE_PROVIDER_NAME))
    print(f"Service provider ID: {SERVICE_PROVIDER_ID}")

In [ ]:
wos_client.service_providers.show()

# Create prod subscription

In [8]:
import os
import json
from datetime import datetime, timezone, timedelta
import time
import uuid
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import WMLCredentialsCP4D
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import wos_sdk_utils as wos_util
import wml_sdk_utils as wml_util

## Create prod subscription

In [32]:
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider - Test"
SUBSCRIPTION_NAME = "cifar monitor wendy"

#MODEL_ASSET_ID = ""
WML_SPACE_ID = "81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7"#"416d5be2-0964-4605-aa0a-e1764a78f916" 

WOS_GUID = "00000000-0000-0000-0000-000000000000"

In [33]:
wos_client = wos_util.get_client()
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)

### 1. Subscribe to Deployment

In [12]:
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest

In [19]:
wos_client.service_providers.show()

None,active,OpenScale Headless Service Provider - Test,custom_machine_learning,2022-02-23 18:40:27.254000+00:00,046a4364-6c34-4a66-ae25-ed83a9f764ec
None,active,OpenScale Headless Service Provider,custom_machine_learning,2022-02-07 19:41:21.084000+00:00,1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1
99999999-9999-9999-9999-999999999999,active,DeepLIIF WML Prod,watson_machine_learning,2022-01-13 19:11:00.897000+00:00,09d6e718-c51f-41e2-9655-3991b6ac0104
99999999-9999-9999-9999-999999999999,active,DeepLIIF WML Pre-Prod,watson_machine_learning,2022-01-13 19:10:38.638000+00:00,1f4fab35-d955-4b91-a1f3-3dab72491116
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML pre_production binding,watson_machine_learning,2022-01-13 06:19:31.996000+00:00,ae0d9dae-8599-400a-bcb9-0b5ab9e004f6


In [17]:
existing_providers_dict = wos_client.service_providers.list().get_result().to_dict()['service_providers']
existing_providers = [sp['entity']['name'] for sp in existing_providers_dict]

SERVICE_PROVIDER_ID = next((sp['metadata']['id'] for sp in existing_providers_dict \
                            if sp['entity']['name'] == SERVICE_PROVIDER_NAME))
print(f"Service provider ID: {SERVICE_PROVIDER_ID}")

Service provider ID: 046a4364-6c34-4a66-ae25-ed83a9f764ec


In [20]:
existing_subscriptions_dict = wos_client.subscriptions.list().get_result().to_dict()['subscriptions']
existing_subscriptions = [sp['entity']['asset']['name'] for sp in existing_subscriptions_dict]
existing_subscriptions

['cifar monitor wendy',
 'deepliif-wendy-test9 Monitor',
 'deepliif-wendy Monitor',
 'DeepLIIF Monitor drew',
 'GermanCreditRiskModelPreProdICP',
 'GermanCreditRiskModelChallengerICP']

In [22]:
existing_subscriptions_dict = wos_client.subscriptions.list().get_result().to_dict()['subscriptions']
existing_subscriptions = [sp['entity']['asset']['name'] for sp in existing_subscriptions_dict]

if not SUBSCRIPTION_NAME in existing_subscriptions:
    ASSET_ID = str(uuid.uuid4())
    ASSET_NAME = SUBSCRIPTION_NAME
    url = ''

    ASSET_DEPLOYMENT_ID = str(uuid.uuid4())
    ASSET_DEPLOYMENT_NAME = SUBSCRIPTION_NAME
    
    subscription_details = wos_client.subscriptions.add(
        data_mart_id=WOS_GUID,
        service_provider_id=SERVICE_PROVIDER_ID,
        asset=Asset(
            asset_id=ASSET_ID,
            name=ASSET_NAME,
            url=url,
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.MULTICLASS_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=ASSET_DEPLOYMENT_ID,
            name=ASSET_DEPLOYMENT_NAME,
            deployment_type= DeploymentTypes.ONLINE
        ),
        asset_properties=AssetPropertiesRequest(
            probability_fields=['probability']
            )
    ).result
    
    SUBSCRIPTION_ID = subscription_details.metadata.id
    print("Subscription ID: {}".format(SUBSCRIPTION_ID))
else:
    SUBSCRIPTION_ID = next((sp['metadata']['id'] for sp in existing_subscriptions_dict \
                            if sp['entity']['asset']['name'] == SUBSCRIPTION_NAME))
    print("Subscription ID: {}".format(SUBSCRIPTION_ID))

Subscription ID: 5c983c3c-6ea7-4c84-9915-988fe259fe52


In [23]:
wos_client.subscriptions.show()

0a0931ad-2ce2-4435-90a8-0ed1467fefcf,cifar monitor wendy,00000000-0000-0000-0000-000000000000,98e120b4-0677-4275-bad7-2423085dc932,cifar monitor wendy,1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1,active,2022-03-31 20:49:19.363000+00:00,5c983c3c-6ea7-4c84-9915-988fe259fe52
59ca2dc0-5155-46aa-b625-3e3d2b823e77,deepliif-wendy-test9 Monitor,00000000-0000-0000-0000-000000000000,1727641c-8d1a-4e0e-8e52-283cc8fd028c,deepliif-wendy-test9 Monitor,1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1,active,2022-03-21 22:05:44.342000+00:00,f359c41d-698e-4efa-9da0-222e8b50ced7
80f88f3e-e7ed-41e0-8546-2a6732dea0e2,deepliif-wendy Monitor,00000000-0000-0000-0000-000000000000,e06982a9-7e72-4d47-9656-6cb7e65bf9a4,deepliif-wendy Monitor,1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1,active,2022-03-18 15:08:03.898000+00:00,54f5e250-d12a-4f9a-89bd-423d0a2faeab
9f2cd22a-15fc-4046-988a-5270a414a331,DeepLIIF Monitor drew,00000000-0000-0000-0000-000000000000,6c562b7a-9a82-4a97-81ce-78857f71de79,DeepLIIF Monitor drew,1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1,active,2022-03-09 18:42:29.239000+00:00,ae81dd6d-19d1-4fae-96aa-b17768261997
6eb7d805-d311-4bc3-878d-5d43edb76ad3,GermanCreditRiskModelPreProdICP,00000000-0000-0000-0000-000000000000,5549aabc-ca61-46ce-ab8a-d7d33cc1b1d5,GermanCreditRiskModelPreProdICP,ae0d9dae-8599-400a-bcb9-0b5ab9e004f6,active,2022-01-13 06:23:13.902000+00:00,3944c36d-c97d-4cbe-9066-d9637f4ec715
a6e676d0-c9c3-4c7a-b1b3-5fdf10e2b22d,GermanCreditRiskModelChallengerICP,00000000-0000-0000-0000-000000000000,641e28cf-ea00-4867-9562-5cb7bfaa281c,GermanCreditRiskModelChallengerICP,ae0d9dae-8599-400a-bcb9-0b5ab9e004f6,active,2022-01-13 06:21:41.540000+00:00,ec94173d-6cfa-4f47-ae04-5fb1497710f5


## 2. Configure Custom Metric Monitors

### Check existence of custom monitor instance


In [38]:
monitor_ids = ['specificity','sensitivity']

In [ ]:
for monitor_id in monitor_ids:
    print('*'*20,'checking',monitor_id,'*'*20)
    print(wos_util.get_monitor_instance(monitor_id,SUBSCRIPTION_ID,wos_client))

In [ ]:
# Openscale MRM service invokes custom metrics deployment url during runtime and wait for the default time of 60 second's to 
# to check the run status ie finished/Failed and fetch the latest measurement. Increase the wait time, if the runtime deployment 
# takes more than 60 seconds to compute and publish the custom metrics 

#Update the wait time here.
custom_metrics_wait_time = 360 #time in seconds <update the time here>

### Create one custom monitor instance per monitor


In [ ]:
for monitor_id in monitor_ids:
    monitor_instance_details = wos_util.monitor_instance_create(monitor_id,
                                                                metadata_deployment,
                                                                metadata_monitor,
                                                                custom_metrics_wait_time,
                                                                wos_client)
    print(monitor_instance_details)

### Manual Evaluation


In [ ]:
import uuid
from pprint import pprint

subscription_id = SUBSCRIPTION_ID
for monitor_id in monitor_ids:
    print('*'*30,monitor_id,'*'*30)
    parameters = {
        "custom_metrics_provider_id": metadata_monitor[monitor_id]['integrated_system_id'],
        "custom_metrics_wait_time":   custom_metrics_wait_time,
        "run_details": {
        "run_id": str(uuid.uuid4()),
        "run_status": "Running"
        }
    }

    payload= {
        "data_mart_id" : WOS_GUID,
        "subscription_id" : subscription_id,
        "custom_monitor_id" : monitor_id,
        "custom_monitor_instance_id" : wos_util.get_monitor_instance(monitor_id,subscription_id,wos_client)['metadata']['id'],
        "custom_monitor_instance_params": parameters

    }

    input_data= { "input_data": [ { "values": payload } ]
                }

    #     print(input_data)

    job_details = wml_client.deployments.score(metadata_monitor[monitor_id]['wml_deployment_id'], input_data)
    pprint(job_details)